# 0.导入相关模块并定义全局变量

In [2]:
from os import listdir
from os.path import isfile,join
import re 
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 200)

#path = r'/home/duhan/data/商品期货1分钟201509csv'
path = r'/home/duhan/data/2013商品期货一分钟数据'
filenamePattern = r'(?P<code>\D+)(?P<num>\d+).(csv|CSV)' 
header=['date', 'time', 'open', 'high', 'low', 'close', 'lot', 'volume']

 ## (1)定义品种目录


In [3]:
InstrumentCatalog = {
    'ZZSRX': 'ZZSR',
    'ZZSRY': 'ZZSR',
}

# 1.读取目录中的所有文件名列表

In [4]:
filenameList = [ filename for filename in listdir(path) if isfile(join(path,filename))  and re.match(filenamePattern, filename)]
filenameDataset = DataFrame(filenameList, columns=['name'])
filenameDataset['code'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('code'))
filenameDataset['num'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('num'))
filenameDataset['catalog'] = filenameDataset['code'].apply(lambda x: InstrumentCatalog.get(x, x))
filenameDataset.head()

,name,code,num,catalog
0,SQZN06.CSV,SQZN,06,SQZN
1,DLI06.CSV,DLI,06,DLI
2,SQZN09.CSV,SQZN,09,SQZN
3,ZZRI03.CSV,ZZRI,03,ZZRI
4,SQBUY06.CSV,SQBUY,06,SQBUY


# 2. 获取所有品种列表

In [5]:
instrumentList  = list(filenameDataset.groupby('catalog').count().index)
print instrumentList
filenameDataset.groupby('catalog').count().head(10)

['DLAX', 'DLAY', 'DLB', 'DLBB', 'DLC', 'DLFB', 'DLI', 'DLJ', 'DLJD', 'DLJM', 'DLL', 'DLM', 'DLP', 'DLV', 'DLY', 'SQAG', 'SQAL', 'SQAU', 'SQBUX', 'SQBUY', 'SQCU', 'SQFU', 'SQPB', 'SQRB', 'SQRU', 'SQWR', 'SQZN', 'ZZCF', 'ZZFG', 'ZZJR', 'ZZME', 'ZZOI', 'ZZPM', 'ZZRI', 'ZZRM', 'ZZRS', 'ZZSR', 'ZZTA', 'ZZTC', 'ZZWHX', 'ZZWHY']


,name,code,num
catalog,,,
DLAX,6,6,6
DLAY,6,6,6
DLB,6,6,6
DLBB,7,7,7
DLC,6,6,6
DLFB,8,8,8
DLI,10,10,10
DLJ,12,12,12
DLJD,7,7,7


# 3. 定义通过文名件导入数据集的方法

In [6]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """    
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [filename[:-4]]*8, df.columns ]
    return df


In [7]:
loadDataFile('DLAX05.CSV').head()

DLAX05                                          
                        date   time  open  high   low close lot volume
2013/01/04 09:01  2013/01/04  09:01  4812  4812  4812  4812   6     58
2013/01/04 09:02  2013/01/04  09:02  4812  4812  4812  4812   0     58
2013/01/04 09:03  2013/01/04  09:03  4812  4812  4812  4812   0     58
2013/01/04 09:04  2013/01/04  09:04  4812  4812  4812  4812   0     58
2013/01/04 09:05  2013/01/04  09:05  4812  4812  4812  4812   0     58

# 4.定义导入一种品种所有数据的函数

In [8]:
def loadInstrumentData(catalog):
    """
    给定顶一个品种代码，导入该品种的所有数据文件
    """
    c1 = filenameDataset['catalog'] == catalog
    df = pd.concat([ loadDataFile(filename) for filename in filenameDataset[ c1 ]['name'] ] , axis=1)    
    df = df.swaplevel(0,1,axis=1)
    df = df.sort_index(axis=1)
    return df

In [9]:
df =loadInstrumentData('DLI').head(10)
df 

close                                                        \
                 DLI03 DLI04 DLI05 DLI06 DLI07 DLI08 DLI09 DLI10 DLI11 DLI12   
2013/10/18 09:01   983   970   981   965   960   950   960   NaN   NaN   NaN   
2013/10/18 09:02   983   970   973   963   960   950   949   NaN   NaN   NaN   
2013/10/18 09:03   984   961   972   965   960   960   943   NaN   NaN   NaN   
2013/10/18 09:04   981   972   969   965   960   960   937   NaN   NaN   NaN   
2013/10/18 09:05   979   962   972   952   960   960   930   NaN   NaN   NaN   
2013/10/18 09:06   978   955   965   960   960   960   930   NaN   NaN   NaN   
2013/10/18 09:07   977   955   964   960   960   936   928   NaN   NaN   NaN   
2013/10/18 09:08   978   958   966   960   960   936   927   NaN   NaN   NaN   
2013/10/18 09:09   978   963   966   960   960   936   924   NaN   NaN   NaN   
2013/10/18 09:10   977   963   967   960   960   936   922   NaN   NaN   NaN   

                        date                                                  \
                       DLI03       DLI04       DLI05       DLI06       DLI07   
2013/10/18 09:01  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:02  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:03  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:04  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:05  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:06  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:07  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:08  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:09  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   
2013/10/18 09:10  2013/10/18  2013/10/18  2013/10/18  2013/10/18  2013/10/18   

                                                            high              \
                       DLI08       DLI09 DLI10 DLI11 DLI12 DLI03 DLI04 DLI05   
2013/10/18 09:01  2013/10/18  2013/10/18   NaN   NaN   NaN   983   970   983   
2013/10/18 09:02  2013/10/18  2013/10/18   NaN   NaN   NaN   984   970   981   
2013/10/18 09:03  2013/10/18  2013/10/18   NaN   NaN   NaN   985   961   974   
2013/10/18 09:04  2013/10/18  2013/10/18   NaN   NaN   NaN   984   972   973   
2013/10/18 09:05  2013/10/18  2013/10/18   NaN   NaN   NaN   982   962   972   
2013/10/18 09:06  2013/10/18  2013/10/18   NaN   NaN   NaN   980   955   972   
2013/10/18 09:07  2013/10/18  2013/10/18   NaN   NaN   NaN   978   955   967   
2013/10/18 09:08  2013/10/18  2013/10/18   NaN   NaN   NaN   980   958   967   
2013/10/18 09:09  2013/10/18  2013/10/18   NaN   NaN   NaN   979   964   968   
2013/10/18 09:10  2013/10/18  2013/10/18   NaN   NaN   NaN   978   963   967   

                                                             lot               \
                 DLI06 DLI07 DLI08 DLI09 DLI10 DLI11 DLI12 DLI03 DLI04  DLI05   
2013/10/18 09:01   965   960   950   975   NaN   NaN   NaN  2704   124  13496   
2013/10/18 09:02   965   960   950   961   NaN   NaN   NaN  1014    12   6942   
2013/10/18 09:03   965   960   960   948   NaN   NaN   NaN   728    42  11134   
2013/10/18 09:04   965   960   960   942   NaN   NaN   NaN   622     2   5168   
2013/10/18 09:05   952   960   960   935   NaN   NaN   NaN  1036    10   6250   
2013/10/18 09:06   960   960   960   935   NaN   NaN   NaN   456     2   5356   
2013/10/18 09:07   960   960   936   929   NaN   NaN   NaN   300     4   3842   
2013/10/18 09:08   960   960   936   928   NaN   NaN   NaN   306     2   3768   
2013/10/18 09:09   960   960   936   928   NaN   NaN   NaN   230    14   2658   
2013/10/18 09:10   960   960   936   925   NaN   NaN   NaN   260     0   4770   

                                                             low              \
                 DLI06 DLI07 DLI08 DLI09 DLI10 DLI11 DLI12 DLI03 DLI04 DLI05   
2

 # 5. 计算加权平均过程

In [10]:
df =loadInstrumentData('ZZSR').head(10)
df 

close                                                  \
                 ZZSRX01 ZZSRX03 ZZSRX05 ZZSRX07 ZZSRX09 ZZSRX11 ZZSRY01   
2013/01/04 09:01    5591    5583    5575     NaN     NaN     NaN    5620   
2013/01/04 09:02    5595    5583    5586     NaN     NaN     NaN    5620   
2013/01/04 09:03    5580    5583    5569     NaN     NaN     NaN    5620   
2013/01/04 09:04    5583    5583    5569     NaN     NaN     NaN    5620   
2013/01/04 09:05    5583    5583    5578     NaN     NaN     NaN    5620   
2013/01/04 09:06    5585    5583    5577     NaN     NaN     NaN    5620   
2013/01/04 09:07    5596    5583    5577     NaN     NaN     NaN    5620   
2013/01/04 09:08    5605    5583    5577     NaN     NaN     NaN    5620   
2013/01/04 09:09    5615    5583    5577     NaN     NaN     NaN    5620   
2013/01/04 09:10    5614    5583    5615     NaN     NaN     NaN    5620   

                                                                date  \
                 ZZSRY03 ZZSRY05 ZZSRY07 ZZSRY09 ZZSRY11     ZZSRX01   
2013/01/04 09:01    5619    5546    5543    5548    5602  2013/01/04   
2013/01/04 09:02    5619    5542    5543    5549    5602  2013/01/04   
2013/01/04 09:03    5619    5545    5543    5551    5602  2013/01/04   
2013/01/04 09:04    5619    5544    5543    5551    5602  2013/01/04   
2013/01/04 09:05    5619    5544    5543    5551    5602  2013/01/04   
2013/01/04 09:06    5619    5552    5543    5559    5602  2013/01/04   
2013/01/04 09:07    5619    5560    5543    5568    5602  2013/01/04   
2013/01/04 09:08    5619    5563    5543    5572    5602  2013/01/04   
2013/01/04 09:09    5619    5573    5543    5582    5602  2013/01/04   
2013/01/04 09:10    5619    5573    5580    5580    5602  2013/01/04   

                                                                              \
                     ZZSRX03     ZZSRX05 ZZSRX07 ZZSRX09 ZZSRX11     ZZSRY01   
2013/01/04 09:01  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:02  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:03  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:04  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:05  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:06  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:07  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:08  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:09  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   
2013/01/04 09:10  2013/01/04  2013/01/04     NaN     NaN     NaN  2013/01/04   

                                                                              \
                     ZZSRY03     ZZSRY05     ZZSRY07     ZZSRY09     ZZSRY11   
2013/01/04 09:01  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:02  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:03  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:04  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:05  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:06  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:07  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:08  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:09  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   
2013/01/04 09:10  2013/01/04  2013/01/04  2013/01/04  2013/01/04  2013/01/04   

                    high                                                  \
                 ZZSRX01 ZZSRX03 ZZSRX05 ZZSRX07 ZZSRX09 ZZSRX11 ZZSRY01   
2013/01/04 09:01    5595    5583    5575     NaN     NaN     NaN    5620   
2013/01/04 09:02    5595    5583    5586     NaN     NaN     NaN    5620   
2013/01/04 09:0

In [11]:
df['close']

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRY01,ZZSRY03,ZZSRY05,ZZSRY07,ZZSRY09,ZZSRY11
2013/01/04 09:01,5591,5583,5575,NaN,NaN,NaN,5620,5619,5546,5543,5548,5602
2013/01/04 09:02,5595,5583,5586,NaN,NaN,NaN,5620,5619,5542,5543,5549,5602
2013/01/04 09:03,5580,5583,5569,NaN,NaN,NaN,5620,5619,5545,5543,5551,5602
2013/01/04 09:04,5583,5583,5569,NaN,NaN,NaN,5620,5619,5544,5543,5551,5602
2013/01/04 09:05,5583,5583,5578,NaN,NaN,NaN,5620,5619,5544,5543,5551,5602
2013/01/04 09:06,5585,5583,5577,NaN,NaN,NaN,5620,5619,5552,5543,5559,5602
2013/01/04 09:07,5596,5583,5577,NaN,NaN,NaN,5620,5619,5560,5543,5568,5602
2013/01/04 09:08,5605,5583,5577,NaN,NaN,NaN,5620,5619,5563,5543,5572,5602
2013/01/04 09:09,5615,5583,5577,NaN,NaN,NaN,5620,5619,5573,5543,5582,5602
2013/01/04 09:10,5614,5583,5615,NaN,NaN,NaN,5620,5619,5573,5580,5580,5602


In [12]:
df['lot'] 

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRY01,ZZSRY03,ZZSRY05,ZZSRY07,ZZSRY09,ZZSRY11
2013/01/04 09:01,60,0,14,NaN,NaN,NaN,0,0,12622,16,4500,0
2013/01/04 09:02,8,0,4,NaN,NaN,NaN,0,0,5288,0,1482,0
2013/01/04 09:03,12,0,6,NaN,NaN,NaN,0,0,3426,0,1638,0
2013/01/04 09:04,4,0,0,NaN,NaN,NaN,0,0,1988,0,578,0
2013/01/04 09:05,22,0,4,NaN,NaN,NaN,0,0,1612,0,842,0
2013/01/04 09:06,54,2,2,NaN,NaN,NaN,0,0,4440,0,1622,0
2013/01/04 09:07,10,0,0,NaN,NaN,NaN,0,0,9404,0,3984,0
2013/01/04 09:08,18,0,0,NaN,NaN,NaN,0,0,6836,0,2750,0
2013/01/04 09:09,16,0,0,NaN,NaN,NaN,0,0,17466,0,3140,0
2013/01/04 09:10,14,0,4,NaN,NaN,NaN,0,0,8966,2,2544,0


In [13]:
df['lot'].sum( axis = 1)

2013/01/04 09:01    17212
2013/01/04 09:02     6782
2013/01/04 09:03     5082
2013/01/04 09:04     2570
2013/01/04 09:05     2480
2013/01/04 09:06     6120
2013/01/04 09:07    13398
2013/01/04 09:08     9604
2013/01/04 09:09    20622
2013/01/04 09:10    11530
dtype: float64

In [14]:
r = df['lot'].div(df['lot'].sum( axis = 1), axis=0)
r

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRY01,ZZSRY03,ZZSRY05,ZZSRY07,ZZSRY09,ZZSRY11
2013/01/04 09:01,0.003486,0.000000,0.000813,NaN,NaN,NaN,0,0,0.733326,0.000930,0.261446,0
2013/01/04 09:02,0.001180,0.000000,0.000590,NaN,NaN,NaN,0,0,0.779711,0.000000,0.218520,0
2013/01/04 09:03,0.002361,0.000000,0.001181,NaN,NaN,NaN,0,0,0.674144,0.000000,0.322314,0
2013/01/04 09:04,0.001556,0.000000,0.000000,NaN,NaN,NaN,0,0,0.773541,0.000000,0.224903,0
2013/01/04 09:05,0.008871,0.000000,0.001613,NaN,NaN,NaN,0,0,0.650000,0.000000,0.339516,0
2013/01/04 09:06,0.008824,0.000327,0.000327,NaN,NaN,NaN,0,0,0.725490,0.000000,0.265033,0
2013/01/04 09:07,0.000746,0.000000,0.000000,NaN,NaN,NaN,0,0,0.701896,0.000000,0.297358,0
2013/01/04 09:08,0.001874,0.000000,0.000000,NaN,NaN,NaN,0,0,0.711787,0.000000,0.286339,0
2013/01/04 09:09,0.000776,0.000000,0.000000,NaN,NaN,NaN,0,0,0.846960,0.000000,0.152265,0
2013/01/04 09:10,0.001214,0.000000,0.000347,NaN,NaN,NaN,0,0,0.777624,0.000173,0.220642,0


In [15]:
df['close']  * r

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRY01,ZZSRY03,ZZSRY05,ZZSRY07,ZZSRY09,ZZSRY11
2013/01/04 09:01,19.489891,0.00000,4.534627,NaN,NaN,NaN,0,0,4067.023704,5.152684,1450.499651,0
2013/01/04 09:02,6.599823,0.00000,3.294603,NaN,NaN,NaN,0,0,4321.158360,0.000000,1212.565320,0
2013/01/04 09:03,13.175915,0.00000,6.574970,NaN,NaN,NaN,0,0,3738.128689,0.000000,1789.165289,0
2013/01/04 09:04,8.689494,0.00000,0.000000,NaN,NaN,NaN,0,0,4288.510506,0.000000,1248.435019,0
2013/01/04 09:05,49.526613,0.00000,8.996774,NaN,NaN,NaN,0,0,3603.600000,0.000000,1884.654032,0
2013/01/04 09:06,49.279412,1.82451,1.822549,NaN,NaN,NaN,0,0,4027.921569,0.000000,1473.316667,0
2013/01/04 09:07,4.176743,0.00000,0.000000,NaN,NaN,NaN,0,0,3902.540678,0.000000,1655.688312,0
2013/01/04 09:08,10.504998,0.00000,0.000000,NaN,NaN,NaN,0,0,3959.669721,0.000000,1595.481050,0
2013/01/04 09:09,4.356512,0.00000,0.000000,NaN,NaN,NaN,0,0,4720.105615,0.000000,849.940840,0
2013/01/04 09:10,6.816652,0.00000,1.947962,NaN,NaN,NaN,0,0,4333.696271,0.967910,1231.181266,0


In [16]:
close = (df['close']  * r).sum(axis=1)
close

2013/01/04 09:01    5546.700558
2013/01/04 09:02    5543.618107
2013/01/04 09:03    5547.044864
2013/01/04 09:04    5545.635019
2013/01/04 09:05    5546.777419
2013/01/04 09:06    5554.164706
2013/01/04 09:07    5562.405732
2013/01/04 09:08    5565.655768
2013/01/04 09:09    5574.402968
2013/01/04 09:10    5574.610061
dtype: float64

In [17]:
df['lot']

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRY01,ZZSRY03,ZZSRY05,ZZSRY07,ZZSRY09,ZZSRY11
2013/01/04 09:01,60,0,14,NaN,NaN,NaN,0,0,12622,16,4500,0
2013/01/04 09:02,8,0,4,NaN,NaN,NaN,0,0,5288,0,1482,0
2013/01/04 09:03,12,0,6,NaN,NaN,NaN,0,0,3426,0,1638,0
2013/01/04 09:04,4,0,0,NaN,NaN,NaN,0,0,1988,0,578,0
2013/01/04 09:05,22,0,4,NaN,NaN,NaN,0,0,1612,0,842,0
2013/01/04 09:06,54,2,2,NaN,NaN,NaN,0,0,4440,0,1622,0
2013/01/04 09:07,10,0,0,NaN,NaN,NaN,0,0,9404,0,3984,0
2013/01/04 09:08,18,0,0,NaN,NaN,NaN,0,0,6836,0,2750,0
2013/01/04 09:09,16,0,0,NaN,NaN,NaN,0,0,17466,0,3140,0
2013/01/04 09:10,14,0,4,NaN,NaN,NaN,0,0,8966,2,2544,0


In [18]:
df['lot'].sum(axis=1)

2013/01/04 09:01    17212
2013/01/04 09:02     6782
2013/01/04 09:03     5082
2013/01/04 09:04     2570
2013/01/04 09:05     2480
2013/01/04 09:06     6120
2013/01/04 09:07    13398
2013/01/04 09:08     9604
2013/01/04 09:09    20622
2013/01/04 09:10    11530
dtype: float64

# 6.定义指标计算函数

In [19]:
def calculateIndex(df):
    """
    通过一个品种的数据全集计算其加权平均
    """
    # 加权平均计算
    calColumns = ['open', 'high', 'low', 'close']
    r =  df['lot'].div(df['lot'].sum( axis = 1), axis=0)
    resultList = [(df[col]  * r).sum(axis=1) for col in calColumns]
    # 求和计算
    sumColumns = ['lot', 'volume']
    resultList += [df[col].sum(axis=1) for col in sumColumns]    
    # 合并数据
    result = pd.concat(resultList, axis=1, keys = calColumns+sumColumns)     
    # 拆分日期和时间
    result['date'] = Series(result.index, index=result.index).apply(lambda x:x.split()[0])
    result['time'] = Series(result.index, index=result.index).apply(lambda x:x.split()[1])
    # 字段排序
    result = result[ ['date', 'time'] + calColumns + sumColumns ]
    return result


In [20]:
result = calculateIndex(df)
#result['date'] = Series(result.index, index=result.index).apply(lambda x:x.split()[0])
#result['time'] = Series(result.index, index=result.index).apply(lambda x:x.split()[1])
result

,date,time,open,high,low,close,lot,volume
2013/01/04 09:01,2013/01/04,09:01,5544.044504,5546.978736,5539.226005,5546.700558,17212,747764
2013/01/04 09:02,2013/01/04,09:02,5545.736951,5548.733412,5542.399587,5543.618107,6782,747742
2013/01/04 09:03,2013/01/04,09:03,5545.396694,5547.067296,5544.377804,5547.044864,5082,748154
2013/01/04 09:04,2013/01/04,09:04,5544.645914,5547.641245,5544.636576,5545.635019,2570,748828
2013/01/04 09:05,2013/01/04,09:05,5546.814516,5547.804032,5545.137903,5546.777419,2480,748352
2013/01/04 09:06,2013/01/04,09:06,5546.930719,5556.888889,5545.214706,5554.164706,6120,746606
2013/01/04 09:07,2013/01/04,09:07,5554.410210,5567.510673,5552.411703,5562.405732,13398,748146
2013/01/04 09:08,2013/01/04,09:08,5562.083090,5566.653894,5559.947730,5565.655768,9604,748920
2013/01/04 09:09,2013/01/04,09:09,5564.555232,5579.246824,5564.402968,5574.402968,20622,754316
2013/01/04 09:10,2013/01/04,09:10,5575.830182,5580.827580,5573.611795,5574.610061,11530,755390


In [21]:
calculateIndex(df)    

,date,time,open,high,low,close,lot,volume
2013/01/04 09:01,2013/01/04,09:01,5544.044504,5546.978736,5539.226005,5546.700558,17212,747764
2013/01/04 09:02,2013/01/04,09:02,5545.736951,5548.733412,5542.399587,5543.618107,6782,747742
2013/01/04 09:03,2013/01/04,09:03,5545.396694,5547.067296,5544.377804,5547.044864,5082,748154
2013/01/04 09:04,2013/01/04,09:04,5544.645914,5547.641245,5544.636576,5545.635019,2570,748828
2013/01/04 09:05,2013/01/04,09:05,5546.814516,5547.804032,5545.137903,5546.777419,2480,748352
2013/01/04 09:06,2013/01/04,09:06,5546.930719,5556.888889,5545.214706,5554.164706,6120,746606
2013/01/04 09:07,2013/01/04,09:07,5554.410210,5567.510673,5552.411703,5562.405732,13398,748146
2013/01/04 09:08,2013/01/04,09:08,5562.083090,5566.653894,5559.947730,5565.655768,9604,748920
2013/01/04 09:09,2013/01/04,09:09,5564.555232,5579.246824,5564.402968,5574.402968,20622,754316
2013/01/04 09:10,2013/01/04,09:10,5575.830182,5580.827580,5573.611795,5574.610061,11530,755390


# 7. 定义完整的计算一个品种的加权平均加并保存到文件

In [22]:
def processInstrument(catalog):
    """
    计算品种的加权平均价，并保存到文件
    """
    df = loadInstrumentData(catalog)
    result = calculateIndex(df)
    filename = 'calculate_index_%s.csv' % catalog
    result.to_csv(join(path, filename), header=False, index=False)

In [23]:
processInstrument('DLI')

# 8. 循环处理所有品种

In [24]:
#for catalog in instrumentList:
#    print catalog,'...........',
#    processInstrument(catalog)
#    print u'完成'

In [35]:
df =loadDataFile('ZZSRX01.CSV')
#df['2013/01/04 09:01']
df.head()

ZZSRX01                                          
                        date   time  open  high   low close lot volume
2013/01/04 09:01  2013/01/04  09:01  5595  5595  5580  5591  60   6702
2013/01/04 09:02  2013/01/04  09:02  5594  5595  5594  5595   8   6708
2013/01/04 09:03  2013/01/04  09:03  5589  5589  5580  5580  12   6702
2013/01/04 09:04  2013/01/04  09:04  5589  5589  5583  5583   4   6702
2013/01/04 09:05  2013/01/04  09:05  5587  5587  5583  5583  22   6694

In [43]:
df.ix['2013/01/04 09:01']

ZZSRX01  date      2013/01/04
         time           09:01
         open            5595
         high            5595
         low             5580
         close           5591
         lot               60
         volume          6702
Name: 2013/01/04 09:01, dtype: object